In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("16P.csv", encoding="ISO-8859-1")
data.drop("Response Id", axis=1, inplace=True)
personality = ["ESTJ", "ENTJ", "ESFJ", "ENFJ", "ISTJ", "ISFJ", "INTJ", "INFJ", "ESTP", "ESFP", "ENTP", "ENFP", "ISTP", "ISFP", "INTP", "INFP"]
values = {}
a = 0
for i in personality:
    values["tp_" + str(i)] = [0]
    values["fp_" + str(i)] = [0]
    values["tn_" + str(i)] = [0]
    values["fn_" + str(i)] = [0]
for i in range(16):
    data.replace(personality[i], i, inplace=True)
data_without_normalization = np.array(data.drop("Personality", axis=1))
data_normalization = data.replace([-3, -2, -1, 0, 1, 2, 3], [0, 1/6, 1/3, 1/2, 2/3, 5/6, 1])
data_normalization = np.array(data_normalization.drop("Personality", axis=1))

In [3]:
def distance_calculation(lower_limit, upper_limit, dataa):
    predictive_data = dataa[lower_limit:upper_limit]
    compared_data = np.delete(dataa, range(lower_limit, upper_limit), axis=0)
    x_squared = np.sum(predictive_data ** 2, axis=1)
    x_times_y = np.matmul(predictive_data, compared_data.T)
    y_squared = np.sum(compared_data ** 2, axis=1)
    distance = (x_squared.reshape(-1, 1) - 2 * x_times_y + y_squared) ** 0.5
    return distance

In [4]:
def knn(lower_limit, upper_limit, k, dataa):
    global a
    distance = distance_calculation(lower_limit, upper_limit, dataa)
    distance_with_sorting = np.sort(distance)
    a += 1
    for person in range(upper_limit - lower_limit):
        location = {}
        for i in range(k):
            location["k" + str(i)] = distance[person].tolist().index(distance_with_sorting[person, i])
        if a == 1:
            for i in list(location.keys()):
                location[i] += 12000
        elif a == 2:
            for i in list(location.keys()):
                if location[i] > 12000:
                    location[i] += 12000
            person += 12000
        elif a == 3:
            for i in list(location.keys()):
                if location[i] > 24000:
                    location[i] += 12000
            person += 24000
        elif a == 4:
            for i in list(location.keys()):
                if location[i] > 36000:
                    location[i] += 12000
            person += 36000
        elif a == 5:
            person += 47999
        list_of_personality = []
        for i in list(location.keys()):
            list_of_personality.append(data.iloc[location[i], -1])
        max_value = list_of_personality.count(data.iloc[location["k0"], -1])
        index = "k0"
        for i in list(location.keys()):
            if list_of_personality.count(data.iloc[location[i], -1]) > max_value:
                max_value = list_of_personality.count(data.iloc[location[i], -1])
                index = i
        if data.iloc[person, -1] == data.iloc[location[index], -1]:
            values["tp_" + personality[data.iloc[location[index], -1]]][0] += 1
            for i in personality:
                if i == personality[data.iloc[location[index], -1]]:
                    continue
                values["tn_" + i][0] += 1
        else:
            values["fp_" + personality[data.iloc[location[index], -1]]][0] += 1
            for i in personality:
                if i == personality[data.iloc[location[index], -1]]:
                    continue
                values["fn_" + i][0] += 1
    return values

In [5]:
def write(message, dataaa):
    global a
    print(message + "\n")
    values_for_k1 = pd.DataFrame(knn(0, 12000, 1, dataaa)) + pd.DataFrame(knn(12000, 24000, 1, dataaa)) + pd.DataFrame(knn(24000, 36000, 1, dataaa)) + pd.DataFrame(knn(36000, 48000, 1, dataaa)) + pd.DataFrame(knn(48000, 59999, 1, dataaa))
    accuracy_for_k1 = 0
    for i in range(16):
        accuracy_for_k1 += (values_for_k1.iloc[0, 4 * i] + values_for_k1.iloc[0, (4 * i) + 2]) / (values_for_k1.iloc[0, 4 * i] + values_for_k1.iloc[0, (4 * i) + 1] + values_for_k1.iloc[0, (4 * i) + 2] + values_for_k1.iloc[0, (4 * i) + 3])
    print(f"Accuracy for k=1 : %{(accuracy_for_k1 / 16) * 100}")
    precision_for_k1 = 0
    for i in range(16):
        precision_for_k1 += values_for_k1.iloc[0, 4 * i] / (values_for_k1.iloc[0, 4 * i] + values_for_k1.iloc[0, (4 * i) + 1])
    print(f"Precision for k=1 : %{(precision_for_k1 / 16) * 100}")
    recall_for_k1 = 0
    for i in range(16):
        recall_for_k1 += values_for_k1.iloc[0, 4 * i] / (values_for_k1.iloc[0, 4 * i] + values_for_k1.iloc[0, (4 * i) + 3])
    print(f"Recall for k=1 : %{(recall_for_k1 / 16) * 100}\n")
    a = 0
    values_for_k3 = pd.DataFrame(knn(0, 12000, 3, dataaa)) + pd.DataFrame(knn(12000, 24000, 3, dataaa)) + pd.DataFrame(knn(24000, 36000, 3, dataaa)) + pd.DataFrame(knn(36000, 48000, 3, dataaa)) + pd.DataFrame(knn(48000, 59999, 3, dataaa))
    accuracy_for_k3 = 0
    for i in range(16):
        accuracy_for_k3 += (values_for_k3.iloc[0, 4 * i] + values_for_k3.iloc[0, (4 * i) + 2]) / (values_for_k3.iloc[0, 4 * i] + values_for_k3.iloc[0, (4 * i) + 1] + values_for_k3.iloc[0, (4 * i) + 2] + values_for_k3.iloc[0, (4 * i) + 3])
    print(f"Accuracy for k=3 : %{(accuracy_for_k3 / 16) * 100}")
    precision_for_k3 = 0
    for i in range(16):
        precision_for_k3 += values_for_k3.iloc[0, 4 * i] / (values_for_k3.iloc[0, 4 * i] + values_for_k3.iloc[0, (4 * i) + 1])
    print(f"Precision for k=3 : %{(precision_for_k3 / 16) * 100}")
    recall_for_k3 = 0
    for i in range(16):
        recall_for_k3 += values_for_k3.iloc[0, 4 * i] / (values_for_k3.iloc[0, 4 * i] + values_for_k3.iloc[0, (4 * i) + 3])
    print(f"Recall for k=3 : %{(recall_for_k3 / 16) * 100}\n")
    a = 0
    values_for_k5 = pd.DataFrame(knn(0, 12000, 5, dataaa)) + pd.DataFrame(knn(12000, 24000, 5, dataaa)) + pd.DataFrame(knn(24000, 36000, 5, dataaa)) + pd.DataFrame(knn(36000, 48000, 5, dataaa)) + pd.DataFrame(knn(48000, 59999, 5, dataaa))
    accuracy_for_k5 = 0
    for i in range(16):
        accuracy_for_k5 += (values_for_k5.iloc[0, 4 * i] + values_for_k5.iloc[0, (4 * i) + 2]) / (values_for_k5.iloc[0, 4 * i] + values_for_k5.iloc[0, (4 * i) + 1] + values_for_k5.iloc[0, (4 * i) + 2] + values_for_k5.iloc[0, (4 * i) + 3])
    print(f"Accuracy for k=5 : %{(accuracy_for_k5 / 16) * 100}")
    precision_for_k5 = 0
    for i in range(16):
        precision_for_k5 += values_for_k5.iloc[0, 4 * i] / (values_for_k5.iloc[0, 4 * i] + values_for_k5.iloc[0, (4 * i) + 1])
    print(f"Precision for k=5 : %{(precision_for_k5 / 16) * 100}")
    recall_for_k5 = 0
    for i in range(16):
        recall_for_k5 += values_for_k5.iloc[0, 4 * i] / (values_for_k5.iloc[0, 4 * i] + values_for_k5.iloc[0, (4 * i) + 3])
    print(f"Recall for k=5 : %{(recall_for_k5 / 16) * 100}\n")
    a = 0
    values_for_k7 = pd.DataFrame(knn(0, 12000, 7, dataaa)) + pd.DataFrame(knn(12000, 24000, 7, dataaa)) + pd.DataFrame(knn(24000, 36000, 7, dataaa)) + pd.DataFrame(knn(36000, 48000, 7, dataaa)) + pd.DataFrame(knn(48000, 59999, 7, dataaa))
    accuracy_for_k7 = 0
    for i in range(16):
        accuracy_for_k7 += (values_for_k7.iloc[0, 4 * i] + values_for_k7.iloc[0, (4 * i) + 2]) / (values_for_k7.iloc[0, 4 * i] + values_for_k7.iloc[0, (4 * i) + 1] + values_for_k7.iloc[0, (4 * i) + 2] + values_for_k7.iloc[0, (4 * i) + 3])
    print(f"Accuracy for k=7 : %{(accuracy_for_k7 / 16) * 100}")
    precision_for_k7 = 0
    for i in range(16):
        precision_for_k7 += values_for_k7.iloc[0, 4 * i] / (values_for_k7.iloc[0, 4 * i] + values_for_k7.iloc[0, (4 * i) + 1])
    print(f"Precision for k=7 : %{(precision_for_k7 / 16) * 100}")
    recall_for_k7 = 0
    for i in range(16):
        recall_for_k7 += values_for_k7.iloc[0, 4 * i] / (values_for_k7.iloc[0, 4 * i] + values_for_k7.iloc[0, (4 * i) + 3])
    print(f"Recall for k=7 : %{(recall_for_k7 / 16) * 100}\n")
    a = 0
    values_for_k9 = pd.DataFrame(knn(0, 12000, 9, dataaa)) + pd.DataFrame(knn(12000, 24000, 9, dataaa)) + pd.DataFrame(knn(24000, 36000, 9, dataaa)) + pd.DataFrame(knn(36000, 48000, 9, dataaa)) + pd.DataFrame(knn(48000, 59999, 9, dataaa))
    accuracy_for_k9 = 0
    for i in range(16):
        accuracy_for_k9 += (values_for_k9.iloc[0, 4 * i] + values_for_k9.iloc[0, (4 * i) + 2]) / (values_for_k9.iloc[0, 4 * i] + values_for_k9.iloc[0, (4 * i) + 1] + values_for_k9.iloc[0, (4 * i) + 2] + values_for_k9.iloc[0, (4 * i) + 3])
    print(f"Accuracy for k=9 : %{(accuracy_for_k9 / 16) * 100}")
    precision_for_k9 = 0
    for i in range(16):
        precision_for_k9 += values_for_k9.iloc[0, 4 * i] / (values_for_k9.iloc[0, 4 * i] + values_for_k9.iloc[0, (4 * i) + 1])
    print(f"Precision for k=9 : %{(precision_for_k9 / 16) * 100}")
    recall_for_k9 = 0
    for i in range(16):
        recall_for_k9 += values_for_k9.iloc[0, 4 * i] / (values_for_k9.iloc[0, 4 * i] + values_for_k9.iloc[0, (4 * i) + 3])
    print(f"Recall for k=9 : %{(recall_for_k9 / 16) * 100}\n")
    a = 0

In [6]:
write("without feature normalization", data_without_normalization)
write("with feature normalization", data_normalization)

without feature normalization

Accuracy for k=1 : %91.72217623431239
Precision for k=1 : %91.72098929805034
Recall for k=1 : %42.484632749436145

Accuracy for k=3 : %84.42084692725325
Precision for k=3 : %84.42013706417008
Recall for k=3 : %26.53814301949973

Accuracy for k=5 : %82.8441170324197
Precision for k=5 : %82.84366762658632
Recall for k=5 : %24.352674616371196

Accuracy for k=7 : %82.17279144876221
Precision for k=7 : %82.1724794267422
Recall for k=7 : %23.505937712733033

Accuracy for k=9 : %81.80233177461393
Precision for k=9 : %81.80209126243409
Recall for k=9 : %23.057887580498374

with feature normalization

Accuracy for k=1 : %81.45691540464314
Precision for k=1 : %81.45665756387365
Recall for k=1 : %22.65175114849997

Accuracy for k=3 : %81.26531273974493
Precision for k=3 : %81.2650782443087
Recall for k=3 : %22.431149966137394

Accuracy for k=5 : %81.15943935956884
Precision for k=5 : %81.15924172349035
Recall for k=5 : %22.31065241874863

Accuracy for k=7 : %81.0834